In [ ]:
!pip install tensorflow-gpu==1.13.1

     |████████████████████████████████| 345.0 MB 4.3 kB/s 


In [ ]:
!pip install tensorflow==1.13.1

     |████████████████████████████████| 92.6 MB 1.4 MB/s 
     |████████████████████████████████| 50 kB 6.2 MB/s 
     |████████████████████████████████| 367 kB 36.7 MB/s 
     |████████████████████████████████| 3.2 MB 32.0 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.7.0
    Uninstalling tensorflow-estimator-2.7.0:
      Successfully uninstalled tensorflow-estimator-2.7.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.7.0
    Uninstalling tensorboard-2.7.0:
      Successfully uninstalled tensorboard-2.7.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.7.0
    Uninstalling tensorflow-2.7.0:
      Successfully uninstalled tensorflow-2.7.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kapre 0.3.6 requires tensorflow>=2.0.0, but you

In [ ]:
import os
import numpy as np
import CNN2Head_input
import tensorflow as tf

import BKNetStyle
from const import *
import time
import math
%load_ext autoreload
%autoreload 2
from copy import copy, deepcopy

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
NUM_CLASSES = 3

def normalise(x):
    return (x-128)/255.0

def load_mini_dataset(index):
    
    label_to_int = {"Mask":0,"Mask_Mouth_Chin":1,"Mask_Chin":2,"Mask_Nose_Mouth":3}
    int_to_label = {0:"Mask",1:"Mask_Mouth_Chin",2:"Mask_Chin",3:"Mask_Nose_Mouth"}


    filename_prefix = '/content/sample_data/train_part'#/content/drive/MyDrive/GT/BigDataProject/train_part0.npy
    filename = filename_prefix + str(index) + ".npy"
                
    X = np.load(filename, allow_pickle=True)
  
    x_mini_data =[]
    y_mini_data=[]
    mask = []    
    mask_prim = 0.0
    mask_sec = 1.0

    for sample in X:
        
        norm_x_prim = normalise(sample[0])

        label_int = sample[1]
        y_prim = [0] * NUM_CLASSES

        if int_to_label[label_int]=="Mask":
          x_mini_data.append(norm_x_prim)
          y_prim[label_int] = 1 # y_ = [1,0,0]
          y_mini_data.append(y_prim)
          mask.append(mask_prim)
          
        else:
          x_mini_data.append(norm_x_prim)
          y_prim[1] = 1 # y_ = [0,1,0]
          y_mini_data.append(y_prim)
          mask.append(mask_prim)

          # create a copy for x & y, add same x and new y for secondary task
          norm_x_sec = deepcopy(norm_x_prim)
          y_sec = [0] * NUM_CLASSES
          y_sec[label_int-1] = 1
          
          x_mini_data.append(norm_x_sec)
          y_mini_data.append(y_sec)

          mask.append(mask_sec)
          

    return x_mini_data, y_mini_data,mask


def create_random_mini_batches(X, Y, mask_mini, mini_batch_size = 32):
    
    m = len(X) # number of training examples
    mini_batches = []
    
    #Reshaping to convert Y to a 2D array from a rank one array
    X = np.array(X)
    Y = np.array(Y)
    Z = np.array(mask_mini)
    
    #Shuffle the data in each of the mini batch
    permutation = list(np.random.permutation(m))
    shuffled_X = X[permutation, :]
    shuffled_Y = Y[permutation, :]
    shuffled_Z = Z[permutation]
    
    n_mini_batches = math.ceil(m / mini_batch_size) # number of mini batches of size mini_batch_size in your partitionning
    for k in range(n_mini_batches):
        
        start_pos = k * mini_batch_size
        end_pos = min(start_pos + mini_batch_size, m)
        
        mini_batch_X = shuffled_X[start_pos : end_pos, :]
        mini_batch_Y = shuffled_Y[start_pos : end_pos, :]
        mini_batch_Z = shuffled_Z[start_pos : end_pos]
        
        mini_batch = (mini_batch_X, mini_batch_Y,mini_batch_Z)
        mini_batches.append(mini_batch)

    return mini_batches


In [ ]:
import numpy as np

def one_hot(indices, num_classes):
    tmp = np.zeros((indices.shape[0], num_classes), dtype=np.float32)
    tmp[np.arange(indices.shape[0]), indices] = 1.0
    return tmp


[  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107]


In [ ]:
sess = tf.compat.v1.InteractiveSession()
global_step = tf.contrib.framework.get_or_create_global_step()

x, y_, mask = BKNetStyle.Input()

y_prim_conv, y_sec_conv, phase_train, keep_prob = BKNetStyle.BKNetModel(x)


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please switch to tf.train.get_or_create_global_step
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [ ]:
prim_loss, sec_loss, l2_loss, loss = BKNetStyle.selective_loss(y_prim_conv, y_sec_conv, y_, mask)

train_step = BKNetStyle.train_op(loss, global_step)

prim_mask = tf.get_collection('prim_mask')[0]
sec_mask = tf.get_collection('sec_mask')[0]

y_prim = tf.get_collection('y_prim')[0]
y_sec = tf.get_collection('y_sec')[0]

prim_correct_prediction = tf.equal(tf.argmax(y_prim_conv, 1), tf.argmax(y_prim, 1))
sec_correct_prediction = tf.equal(tf.argmax(y_sec_conv, 1), tf.argmax(y_sec, 1))

prim_true_pred = tf.reduce_sum(tf.cast(prim_correct_prediction, dtype=tf.float32) * prim_mask)
sec_true_pred = tf.reduce_sum(tf.cast(sec_correct_prediction, dtype=tf.float32) * sec_mask)

saver = tf.train.Saver()

if not os.path.isfile(SAVE_FOLDER + 'model.ckpt.index'):
    print('Create new model')
    sess.run(tf.global_variables_initializer())
    print('OK')
else:
    print('Restoring existed model')
    saver.restore(sess, SAVE_FOLDER + 'model.ckpt')
    print('OK')

loss_summary_placeholder = tf.placeholder(tf.float32)
tf.summary.scalar('loss', loss_summary_placeholder)
merge_summary = tf.summary.merge_all()
writer = tf.summary.FileWriter("./summary/")

learning_rate = tf.get_collection('learning_rate')[0]

current_epoch = (int)(global_step.eval() / (1000 // BATCH_SIZE)) 
for epoch in range(current_epoch + 1, NUM_EPOCHS):
    tic_epoch = time.time()
    print('Epoch:', str(epoch))

    avg_ttl = []
    avg_rgl = []
    avg_prim_loss = []
    avg_sec_loss = []

    prim_nb_true_pred = 0
    sec_nb_true_pred = 0

    prim_nb_train = 0
    sec_nb_train = 0

    print("Learning rate: %f" % learning_rate.eval())
       #lazy loading of dataset and training
    n_mini_batches = 0
    epoch_cost = 0
    total_time_taken_to_load_dataset = 0
    total_time_taken_to_create_mini_batches = 0            
    tic = time.time()
    #file_indices = get_shuffled_indices(1, 26) # todo change to total part files
    file_indices = np.arange(0, 61) #[0, 1, 2] #TODO - to remove

    mini_batch_size = BATCH_SIZE

    for j in file_indices:
        print("file ", j)
        X_mini = None
        Y_mini = None

        lmd_tic = time.time()
        X_mini, Y_mini, mask_mini = load_mini_dataset(j)
        lmd_toc = time.time()
        total_time_taken_to_load_dataset += (lmd_toc-lmd_tic)
        #print(total_time_taken_to_load_dataset)
        
        rmb_tic = time.time()
        mini_batches = None
        mini_batches = create_random_mini_batches(X_mini, Y_mini, mask_mini, mini_batch_size = mini_batch_size) # todo Samarth add for mask
        rmb_toc = time.time()
        total_time_taken_to_create_mini_batches += (rmb_toc-rmb_tic)
        
        #bat_ind = 0
        for mini_batch in mini_batches:
            #print("at minibatch ", bat_ind)
            #bat_ind+=1
            batch_img = mini_batch[0] 
            batch_label = mini_batch[1]
            batch_mask = mini_batch[2]
            
            curr_batch_size = batch_img.shape[0]

            for i in range(curr_batch_size):
                if batch_mask[i] == 0.0:
                    prim_nb_train += 1
                else:
                    if batch_mask[i] == 1.0:
                        sec_nb_train += 1                        

            batch_img = CNN2Head_input.augmentation(batch_img, 48)

            ttl, pml, sel, l2l, _ = sess.run([loss, prim_loss, sec_loss, l2_loss, train_step],
                                                  feed_dict={x: batch_img, y_: batch_label, mask: batch_mask,
                                                            phase_train: True,
                                                            keep_prob: 0.5})

            prim_nb_true_pred += sess.run(prim_true_pred, feed_dict={x: batch_img, y_: batch_label, mask: batch_mask,
                                                                      phase_train: True,
                                                                      keep_prob: 0.5})

            sec_nb_true_pred += sess.run(sec_true_pred,
                                            feed_dict={x: batch_img, y_: batch_label, mask: batch_mask,
                                                      phase_train: True,
                                                      keep_prob: 0.5})


            avg_ttl.append(ttl)
            avg_prim_loss.append(pml)
            avg_sec_loss.append(sel)
            avg_rgl.append(l2l)

    prim_train_accuracy = prim_nb_true_pred * 1.0 / prim_nb_train
    sec_train_accuracy = sec_nb_true_pred * 1.0 / sec_nb_train

    avg_prim_loss = np.average(avg_prim_loss)
    avg_sec_loss = np.average(avg_sec_loss)

    avg_rgl = np.average(avg_rgl)
    avg_ttl = np.average(avg_ttl)
    tic_epoch2 = time.time()
    print(tic_epoch2 - tic_epoch)
    
    with open('log.csv', 'w+') as f:
        f.write('{0},{1},{2},{3},{4},{5},{6}\n'.format(current_epoch, prim_train_accuracy, sec_train_accuracy, avg_prim_loss, avg_sec_loss, avg_ttl, avg_rgl))

    print('Primary task train accuracy: ' + str(prim_train_accuracy * 100))
    print('Secondary task train accuracy: ' + str(sec_train_accuracy * 100))

    print('Total loss: ' + str(avg_ttl) + '. L2-loss: ' + str(avg_rgl))
    print('Primary loss: ' + str(avg_prim_loss))
    print('Secondary loss: ' + str(avg_sec_loss))
    
    print('\n')

    saver.save(sess, 'sample_data/' + 'model.ckpt')

Create new model
OK
Epoch: 1
Learning rate: 0.010000
file  0
file  1
file  2
file  3
file  4
file  5
file  6
file  7
file  8
file  9
file  10
file  11
file  12
file  13
file  14
file  15
file  16
file  17
file  18
file  19
file  20
file  21
file  22
file  23
file  24
file  25
file  26
file  27
file  28
file  29
file  30
file  31
file  32
file  33
file  34
file  35
file  36
file  37
file  38
file  39
file  40
file  41
file  42
file  43
file  44
file  45
file  46
file  47
file  48
file  49
file  50
file  51
file  52
file  53
file  54
file  55
file  56
file  57
file  58
file  59
file  60
235.29842734336853
Primary task train accuracy: 92.07868852459016
Secondary task train accuracy: 87.09729958630574
Total loss: 2.2310944. L2-loss: 1.6503676
Primary loss: 0.22532612
Secondary loss: 0.35540077


Epoch: 2
Learning rate: 0.010000
file  0
file  1
file  2
file  3
file  4
file  5
file  6
file  7
file  8
file  9
file  10
file  11
file  12
file  13
file  14
file  15
file  16
file  17
file  18
fil

KeyboardInterrupt: ignored

In [ ]:
#debug
 
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    print(local_device_protos)
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

print(get_available_gpus())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 13808043833954068218
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 272769565268015323
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 2850385044571302443
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11338832282
locality {
  bus_id: 1
  links {
  }
}
incarnation: 2516489293699904995
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"
]
['/device:GPU:0']
